## Склеиватель файлов на hdfs
FileGraphGluing - склеиватель файлов на hdfs можно применять к таблице (или списку таблиц) в личной директории или к директории с полным доступом, к одной партиции таблицы с подпартициями, к одной партиции с файлами.    
Если известно, что в директории нет подпапок можно применить подфункцию: .gluing_for_file_path(string_path)   
Вывести файлы в директории: get_list_files(path)  
Вывести папки в директории: get_list_dir(path)  

In [ ]:
import os
import sys
import getpass
import datetime
import numpy as np
import pandas as pd
from pyspark.sql import functions as f

In [ ]:
# импортируем объекты из модулей model_pipe
from utils import build_spark, run_cmd
from spark_gluer import FileGraphGluing

In [ ]:
import pyspark.sql.utils as pysut
from pyspark.sql.functions import col

In [ ]:
from typing import Callable, List, Dict

In [ ]:
import logging
logging.basicConfig(
    format="%(asctime)s %(levelname)s %(message)s",
    level=logging.DEBUG,
    stream=sys.stdout,
)

In [ ]:
os.environ["PYSPARK_DRIVER_PYTHON"] = 'python3.6' 

spark = build_spark(
    app_name='spark_gluing', 
    user='username',
    spark_home = '',
    params = {
        'spark.scheduler.mode': 'FIFO',
        'spark.driver.memory': '20g', 
        'spark.driver.maxResultSize': '19g',
        'spark.executor.memory': '30g',
        'spark.executor.cores': '5',
        'spark.dynamicAllocation.maxExecutors': '70',
        'spark.sql.shuffle.partitions': '700',
        'spark.yarn.executor.memoryOverhead': '5g'
}
)

In [ ]:
spark

In [ ]:
sdf = spark.table("default.fs_test_joined_df")
sdf.repartition(1000).write.mode('overwrite').format('parquet').saveAsTable("default.fs_test_gluer_joined_df")

In [ ]:
string_path = "hdfs://.../fs_test_gluer_joined_df"

In [ ]:
check = FileGraphGluing(spark, 128, 'parquet', True) # если таблица большая логи лучше указывать False, так как может зависнуть wrkon
check.glue_directories_recursively(string_path, '')

In [ ]:
default_tables = """HADOOP_USER_NAME=hdfs hdfs dfs -count /.../fs_test_gluer_joined_df | sort -k2 -nr | awk '{if ($2 > 0) print int($2) "\t" int($3 / (1024*1024))" Mb\t" ($3/(1024*1024*$2)) "\t" $4}'"""
out = run_cmd(default_tables, 'files check', True)

In [ ]:
spark.sql('drop table if exists default.fs_test_gluer_joined_df')

In [ ]:
check.graph_node

In [ ]:
spark.stop()

### Команды hdfs

In [ ]:
cmd_dirs = """
HADOOP_USER_NAME=hdfs hdfs dfs -count /user/* | egrep -w -v "yarn|hive|spark" | sort -k2 -nr | 
awk '{ if ($2 > 1000) print int($2) "\t\t" int($3 / (1024*1024))" Mb\t\t" ($3/(1024*1024*$2)) " Mb \t \t" $4}'
"""

s_output_dirs, s_err = run_cmd(cmd_dirs, 'files check')

In [ ]:
s_output_dirs = change_cmd_output(s_output_dirs, 'dirs')
s_output_dirs.head()

In [ ]:
mail_name = ""
mail_pass = getpass.getpass()

send_email_old(mail_name, mail_pass, s_output_default, "files in default")